In [1]:
import qkeras as q
import tensorflow as tf
from tensorflow import keras
from qkeras import utils as u
import numpy as np
import larq as lq
from tqdm import tqdm
import cv2 
import os
import json

In [2]:
path_quicknet = "../../quicknet_weights.h5"
path_quicknet_small = "../../quicknet_small_weights.h5"
path_quicknet_large = "../../quicknet_large_weights.h5"

In [3]:
# create random dataset
test_data = np.random.randint(low=0, high=254, size=(128, 1, 224, 224, 3))

# load data from ImageNET dataset
def loadImageNetData(path, image_num):
    file_list = os.listdir(path)
    file_list.sort()
    print(len(file_list))
    file_list = file_list[0:image_num]
    im = []
    for i in tqdm(file_list):
        img = cv2.imread(os.path.join(path, i))
        img = cv2.resize(img, (224, 224))
        im.append(img)
    im = np.asarray(im)
    im = np.expand_dims(im, axis=1)
    return im

In [4]:
def add_qkeras_residual(model, filter_index):
    model.add(q.QActivation("binary"))
    model.add(q.QConv2D(filters[filter_index], (3, 3), activation="relu", kernel_quantizer="binary",
                kernel_initializer="glorot_normal", padding="same", use_bias=False))
    model.add(tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5))
    return model

def add_qkeras_transistion(model, strides, filter_index):
    model.add(tf.keras.layers.Activation("relu"))
    model.add(tf.keras.layers.MaxPool2D(pool_size=strides, strides=1))
    model.add(tf.keras.layers.DepthwiseConv2D((3, 3), padding="same",
                strides=strides, trainable=False, use_bias=False))
    model.add(q.QConv2D(filters[filter_index], (1, 1), kernel_initializer="glorot_normal", use_bias=False))
    model.add(tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5))
    return model

In [5]:
def larq_add_qkeras_residual(model, filter_index):
    model.add(q.QActivation("binary"))
    model.add(lq.layers.QuantConv2D(filters[filter_index], (3, 3), activation="relu", kernel_quantizer=lq.quantizers.SteSign(clip_value=1.25),
                kernel_constraint=lq.constraints.WeightClip(clip_value=1.25), kernel_initializer="glorot_normal", padding="same", use_bias=False))
    model.add(tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5))
    return model

def larq_add_qkeras_transistion(model, strides, filter_index):
    model.add(tf.keras.layers.Activation("relu"))
    model.add(tf.keras.layers.MaxPool2D(pool_size=strides, strides=1))
    model.add(tf.keras.layers.DepthwiseConv2D((3, 3), padding="same",
                strides=strides, trainable=False, use_bias=False))
    model.add(lq.layers.QuantConv2D(filters[filter_index], (1, 1), kernel_initializer="glorot_normal", use_bias=False))
    model.add(tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5))
    return model

In [6]:
filters = ((64, 128, 256, 512))
qkeras_quickNet = tf.keras.models.Sequential() 

# INPUT
qkeras_quickNet.add(tf.keras.layers.InputLayer(input_shape=(224, 224, 3)))

# STEM MODULE
qkeras_quickNet.add(q.QConv2D(filters[0] // 4, (3, 3), kernel_initializer="he_normal", padding="same", strides=2, use_bias=False))
qkeras_quickNet.add(tf.keras.layers.BatchNormalization())
qkeras_quickNet.add(tf.keras.layers.Activation("relu"))
qkeras_quickNet.add(q.QDepthwiseConv2D((3, 3),padding="same",strides=2, use_bias=False))
qkeras_quickNet.add(tf.keras.layers.BatchNormalization(scale=False, center=False))
qkeras_quickNet.add(q.QConv2D(filters[0], 1, kernel_initializer="he_normal", use_bias=False))
qkeras_quickNet.add(tf.keras.layers.BatchNormalization())

for i in range(0,3):
    # RESIDUAL MODULE
    for j in range(0,4):
        add_qkeras_residual(qkeras_quickNet, filter_index=i)

    # TRANSITION MODULE
    add_qkeras_transistion(qkeras_quickNet, strides=2, filter_index=i+1)
    
for i in range(0,4):
        add_qkeras_residual(qkeras_quickNet, filter_index=3)
        
# FINAL
qkeras_quickNet.add(tf.keras.layers.Activation("relu"))
qkeras_quickNet.add(tf.keras.layers.AveragePooling2D(pool_size=(7,7)))
qkeras_quickNet.add(tf.keras.layers.Flatten())
qkeras_quickNet.add(q.QDense(1000, kernel_initializer="glorot_normal"))
qkeras_quickNet.add(tf.keras.layers.Activation("softmax", dtype="float32"))


In [7]:
filters = ((64, 128, 256, 512))
larq_quickNet = tf.keras.models.Sequential() 

# INPUT
larq_quickNet.add(tf.keras.layers.InputLayer(input_shape=(224, 224, 3)))

# STEM MODULE
larq_quickNet.add(lq.layers.QuantConv2D(filters[0] // 4, (3, 3), kernel_initializer="he_normal", padding="same", strides=2, use_bias=False))
larq_quickNet.add(tf.keras.layers.BatchNormalization())
larq_quickNet.add(tf.keras.layers.Activation("relu"))
larq_quickNet.add(lq.layers.QuantDepthwiseConv2D((3, 3),padding="same",strides=2, use_bias=False))
larq_quickNet.add(tf.keras.layers.BatchNormalization(scale=False, center=False))
larq_quickNet.add(lq.layers.QuantConv2D(filters[0], 1, kernel_initializer="he_normal", use_bias=False))
larq_quickNet.add(tf.keras.layers.BatchNormalization())

for i in range(0,3):
    # RESIDUAL MODULE
    for j in range(0,4):
        larq_add_qkeras_residual(larq_quickNet, filter_index=i)

    # TRANSITION MODULE
    larq_add_qkeras_transistion(larq_quickNet, strides=2, filter_index=i+1)
    
for i in range(0,4):
        larq_add_qkeras_residual(larq_quickNet, filter_index=3)
        
# FINAL
larq_quickNet.add(tf.keras.layers.Activation("relu"))
larq_quickNet.add(tf.keras.layers.AveragePooling2D(pool_size=(7,7)))
larq_quickNet.add(tf.keras.layers.Flatten())
larq_quickNet.add(lq.layers.QuantDense(1000, kernel_initializer="glorot_normal"))
larq_quickNet.add(tf.keras.layers.Activation("softmax", dtype="float32"))


In [8]:
filters = ((32, 64, 256, 512))
qkeras_quickNet_small = tf.keras.models.Sequential() 

# INPUT
qkeras_quickNet_small.add(tf.keras.layers.InputLayer(input_shape=(224, 224, 3)))

# STEM MODULE
qkeras_quickNet_small.add(q.QConv2D(filters[0] // 4, (3, 3), kernel_initializer="he_normal", padding="same", strides=2, use_bias=False))
qkeras_quickNet_small.add(tf.keras.layers.BatchNormalization())
qkeras_quickNet_small.add(tf.keras.layers.Activation("relu"))
qkeras_quickNet_small.add(q.QDepthwiseConv2D((3, 3),padding="same",strides=2, use_bias=False))
qkeras_quickNet_small.add(tf.keras.layers.BatchNormalization(scale=False, center=False))
qkeras_quickNet_small.add(q.QConv2D(filters[0], 1, kernel_initializer="he_normal", use_bias=False))
qkeras_quickNet_small.add(tf.keras.layers.BatchNormalization())

for i in range(0,3):
    # RESIDUAL MODULE
    for j in range(0,4):
        add_qkeras_residual(qkeras_quickNet_small, filter_index=i)

    # TRANSITION MODULE
    add_qkeras_transistion(qkeras_quickNet_small, strides=2, filter_index=i+1)
    
for i in range(0,4):
        add_qkeras_residual(qkeras_quickNet_small, filter_index=3)
        
# FINAL
qkeras_quickNet_small.add(tf.keras.layers.Activation("relu"))
qkeras_quickNet_small.add(tf.keras.layers.AveragePooling2D(pool_size=(7,7)))
qkeras_quickNet_small.add(tf.keras.layers.Flatten())
qkeras_quickNet_small.add(q.QDense(1000, kernel_initializer="glorot_normal"))
qkeras_quickNet_small.add(tf.keras.layers.Activation("softmax", dtype="float32"))

In [9]:
filters = ((32, 64, 256, 512))
larq_quickNet_small = tf.keras.models.Sequential() 

# INPUT
larq_quickNet_small.add(tf.keras.layers.InputLayer(input_shape=(224, 224, 3)))

# STEM MODULE
larq_quickNet_small.add(lq.layers.QuantConv2D(filters[0] // 4, (3, 3), kernel_initializer="he_normal", padding="same", strides=2, use_bias=False))
larq_quickNet_small.add(tf.keras.layers.BatchNormalization())
larq_quickNet_small.add(tf.keras.layers.Activation("relu"))
larq_quickNet_small.add(lq.layers.QuantDepthwiseConv2D((3, 3),padding="same",strides=2, use_bias=False))
larq_quickNet_small.add(tf.keras.layers.BatchNormalization(scale=False, center=False))
larq_quickNet_small.add(lq.layers.QuantConv2D(filters[0], 1, kernel_initializer="he_normal", use_bias=False))
larq_quickNet_small.add(tf.keras.layers.BatchNormalization())

for i in range(0,3):
    # RESIDUAL MODULE
    for j in range(0,4):
        larq_add_qkeras_residual(larq_quickNet_small, filter_index=i)

    # TRANSITION MODULE
    larq_add_qkeras_transistion(larq_quickNet_small, strides=2, filter_index=i+1)
    
for i in range(0,4):
        larq_add_qkeras_residual(larq_quickNet_small, filter_index=3)
        
# FINAL
larq_quickNet_small.add(tf.keras.layers.Activation("relu"))
larq_quickNet_small.add(tf.keras.layers.AveragePooling2D(pool_size=(7,7)))
larq_quickNet_small.add(tf.keras.layers.Flatten())
larq_quickNet_small.add(lq.layers.QuantDense(1000, kernel_initializer="glorot_normal"))
larq_quickNet_small.add(tf.keras.layers.Activation("softmax", dtype="float32"))


In [10]:
filters = ((64, 128, 256, 512))
qkeras_quickNet_large = tf.keras.models.Sequential() 

# INPUT
qkeras_quickNet_large.add(tf.keras.layers.InputLayer(input_shape=(224, 224, 3)))

# STEM MODULE
qkeras_quickNet_large.add(q.QConv2D(filters[0] // 4, (3, 3), kernel_initializer="he_normal", padding="same", strides=2, use_bias=False))
qkeras_quickNet_large.add(tf.keras.layers.BatchNormalization())
qkeras_quickNet_large.add(tf.keras.layers.Activation("relu"))
qkeras_quickNet_large.add(q.QDepthwiseConv2D((3, 3),padding="same",strides=2, use_bias=False))
qkeras_quickNet_large.add(tf.keras.layers.BatchNormalization(scale=False, center=False))
qkeras_quickNet_large.add(q.QConv2D(filters[0], 1, kernel_initializer="he_normal", use_bias=False))
qkeras_quickNet_large.add(tf.keras.layers.BatchNormalization())

for i in range(0,6):
    add_qkeras_residual(qkeras_quickNet_large, filter_index=0)
add_qkeras_transistion(qkeras_quickNet_large, strides=2, filter_index=1)

for i in range(0,8):
    add_qkeras_residual(qkeras_quickNet_large, filter_index=1)
add_qkeras_transistion(qkeras_quickNet_large, strides=2, filter_index=2)

for i in range(0,12):
    add_qkeras_residual(qkeras_quickNet_large, filter_index=2)
add_qkeras_transistion(qkeras_quickNet_large, strides=2, filter_index=3)

for i in range(0,6):
    add_qkeras_residual(qkeras_quickNet_large, filter_index=3)
        
# FINAL
qkeras_quickNet_large.add(tf.keras.layers.Activation("relu"))
qkeras_quickNet_large.add(tf.keras.layers.AveragePooling2D(pool_size=(7,7)))
qkeras_quickNet_large.add(tf.keras.layers.Flatten())
qkeras_quickNet_large.add(q.QDense(1000, kernel_initializer="glorot_normal"))
qkeras_quickNet_large.add(tf.keras.layers.Activation("softmax", dtype="float32"))

In [11]:
filters = ((64, 128, 256, 512))
larq_quickNet_large = tf.keras.models.Sequential() 

# INPUT
larq_quickNet_large.add(tf.keras.layers.InputLayer(input_shape=(224, 224, 3)))

# STEM MODULE
larq_quickNet_large.add(lq.layers.QuantConv2D(filters[0] // 4, (3, 3), kernel_initializer="he_normal", padding="same", strides=2, use_bias=False))
larq_quickNet_large.add(tf.keras.layers.BatchNormalization())
larq_quickNet_large.add(tf.keras.layers.Activation("relu"))
larq_quickNet_large.add(lq.layers.QuantDepthwiseConv2D((3, 3),padding="same",strides=2, use_bias=False))
larq_quickNet_large.add(tf.keras.layers.BatchNormalization(scale=False, center=False))
larq_quickNet_large.add(lq.layers.QuantConv2D(filters[0], 1, kernel_initializer="he_normal", use_bias=False))
larq_quickNet_large.add(tf.keras.layers.BatchNormalization())

for i in range(0,6):
    larq_add_qkeras_residual(larq_quickNet_large, filter_index=0)
larq_add_qkeras_transistion(larq_quickNet_large, strides=2, filter_index=1)

for i in range(0,8):
    larq_add_qkeras_residual(larq_quickNet_large, filter_index=1)
larq_add_qkeras_transistion(larq_quickNet_large, strides=2, filter_index=2)

for i in range(0,12):
    larq_add_qkeras_residual(larq_quickNet_large, filter_index=2)
larq_add_qkeras_transistion(larq_quickNet_large, strides=2, filter_index=3)

for i in range(0,6):
    larq_add_qkeras_residual(larq_quickNet_large, filter_index=3)
        
# FINAL
larq_quickNet_large.add(tf.keras.layers.Activation("relu"))
larq_quickNet_large.add(tf.keras.layers.AveragePooling2D(pool_size=(7,7)))
larq_quickNet_large.add(tf.keras.layers.Flatten())
larq_quickNet_large.add(lq.layers.QuantDense(1000, kernel_initializer="glorot_normal"))
larq_quickNet_large.add(tf.keras.layers.Activation("softmax", dtype="float32"))

In [12]:
#load weights for qkeras networks
qkeras_quickNet.load_weights(path_quicknet)
qkeras_quickNet_small.load_weights(path_quicknet_small)
qkeras_quickNet_large.load_weights(path_quicknet_large)

#load weights for larq networks
larq_quickNet.load_weights(path_quicknet)
larq_quickNet_small.load_weights(path_quicknet_small)
larq_quickNet_large.load_weights(path_quicknet_large)

In [13]:
def calculate_MSE(res_qkeras, res_larq):
    qres = np.asarray(res_qkeras)
    lres = np.asarray(res_larq)    
    qres = np.squeeze(qres)
    lres = np.squeeze(lres)
    mse = []
    for i in range(0,qres.shape[0]):
        real = lres[i,:]
        pred = qres[i,:]
        mse.append(((real - pred)**2).mean())
    return mse

def calculateAbsoluteError(res_qkeras, res_larq):
    error_counter = 0
    for i, j in zip(res_qkeras, res_larq):
        pred = np.argmax(np.asarray(i))
        real = np.argmax(np.asarray(j))
    if pred != real:
        error_counter += 1
    return error_counter

In [14]:
# uncomment following line to test with imageNET data
#test_data = loadImageNetData("../../ILSVRC2012_img_val", 48000)

In [15]:
res_qkeras = []
res_larq = []

sample_num = 2

for i in tqdm(range(0,sample_num)):
    res_qkeras.append(qkeras_quickNet.predict(test_data[i,:,:,:]))
    res_larq.append(larq_quickNet.predict(test_data[i,:,:,:]))   
mse = calculate_MSE(res_qkeras, res_larq)
print("\n\nMSE for quicknet -> ", np.asarray(mse).mean())
print("Absolute errors for quicknet -> ", calculateAbsoluteError(res_qkeras, res_larq))

res_qkeras = []
res_larq = []
for i in tqdm(range(0,sample_num)):
    res_qkeras.append(qkeras_quickNet_small.predict(test_data[i,:,:,:]))
    res_larq.append(larq_quickNet_small.predict(test_data[i,:,:,:]))    
mse = calculate_MSE(res_qkeras, res_larq)
print("\n\nMSE for quicknet_small -> ", np.asarray(mse).mean())
print("Absolute errors for quicknet_small -> ", calculateAbsoluteError(res_qkeras, res_larq))

res_qkeras = []
res_larq = []
for i in tqdm(range(0,sample_num)):
    res_qkeras.append(qkeras_quickNet_large.predict(test_data[i,:,:,:]))
    res_larq.append(larq_quickNet_large.predict(test_data[i,:,:,:]))
mse = calculate_MSE(res_qkeras, res_larq)
print("\n\nMSE for quicknet_large -> ", np.asarray(mse).mean())
print("Absolute errors for quicknet_large -> ", calculateAbsoluteError(res_qkeras, res_larq))

  0%|          | 0/2 [00:00<?, ?it/s]



MSE for quicknet ->  7.661192e-05
Absolute errors for quicknet ->  0


  0%|          | 0/2 [00:00<?, ?it/s]



MSE for quicknet_small ->  4.548372e-05
Absolute errors for quicknet_small ->  0


100%|██████████| 2/2 [00:04<00:00,  2.43s/it]



MSE for quicknet_large ->  0.00035409242
Absolute errors for quicknet_large ->  0


In [16]:
# save all models in topology (.txt), weights (.h5) format
with lq.context.quantized_scope(True):
    larq_quickNet.save("./larq_models/larq_quickNet_weights.h5")  
    larq_quickNet_small.save("./larq_models/larq_quickNet_small_weights.h5")
    larq_quickNet_large.save("./larq_models/larq_quickNet_large_weights.h5")
    
larq_quickNet = larq_quickNet.to_json()
larq_quickNet_small = larq_quickNet_small.to_json()
larq_quickNet_large = larq_quickNet_large.to_json()

with open('./larq_models/larq_quickNet.txt', 'w') as outfile:
    json.dump(larq_quickNet, outfile)
with open('./larq_models/larq_quickNet_small.txt', 'w') as outfile:
    json.dump(larq_quickNet_small, outfile)
with open('./larq_models/larq_quickNet_large.txt', 'w') as outfile:
    json.dump(larq_quickNet_large, outfile)


u.model_save_quantized_weights(qkeras_quickNet, filename="./qkeras_models/qkeras_quickNet_weights.h5")
u.model_save_quantized_weights(qkeras_quickNet_small, filename="./qkeras_models/qkeras_quickNet_small_weights.h5")
u.model_save_quantized_weights(qkeras_quickNet_large, filename="./qkeras_models/qkeras_quickNet_large_weights.h5")

qkeras_quickNet = qkeras_quickNet.to_json()
qkeras_quickNet_small = qkeras_quickNet_small.to_json()
qkeras_quickNet_large = qkeras_quickNet_large.to_json()

with open('./qkeras_models/qkeras_quickNet.txt', 'w') as outfile:
    json.dump(qkeras_quickNet, outfile)
with open('./qkeras_models/qkeras_quickNet_small.txt', 'w') as outfile:
    json.dump(qkeras_quickNet_small, outfile)
with open('./qkeras_models/qkeras_quickNet_large.txt', 'w') as outfile:
    json.dump(qkeras_quickNet_large, outfile)

... quantizing model
  batch_normalization has not been quantized
  batch_normalization_1 has not been quantized
  batch_normalization_2 has not been quantized
  batch_normalization_3 has not been quantized
  batch_normalization_4 has not been quantized
  batch_normalization_5 has not been quantized
  batch_normalization_6 has not been quantized
  depthwise_conv2d has not been quantized
  batch_normalization_7 has not been quantized
  batch_normalization_8 has not been quantized
  batch_normalization_9 has not been quantized
  batch_normalization_10 has not been quantized
  batch_normalization_11 has not been quantized
  depthwise_conv2d_1 has not been quantized
  batch_normalization_12 has not been quantized
  batch_normalization_13 has not been quantized
  batch_normalization_14 has not been quantized
  batch_normalization_15 has not been quantized
  batch_normalization_16 has not been quantized
  depthwise_conv2d_2 has not been quantized
  batch_normalization_17 has not been quantiz